In [1]:
%cd ..

/home/aymeric/Documents/DAC/NeuralNetworksDIY


In [2]:
from src.convolution import *
from utils.mltools import *

def load_usps(fn):
    with open(fn, "r") as f:
        f.readline()
        data = [[float(X) for X in l.split()] for l in f if len(l.split()) > 2]
    tmp = np.array(data)
    return tmp[:, 1:], tmp[:, 0].astype(int)


def get_usps(l, X, y):
    if type(l) != list:
        resx = X[y == l, :]
        resy = y[y == l]
        return resx, resy
    tmp = list(zip(*[get_usps(i, X, y) for i in l]))
    tmpx, tmpy = np.vstack(tmp[0]), np.hstack(tmp[1])
    return tmpx, tmpy

alltrainx, alltrainy = load_usps("data/USPS_train.txt")
alltestx, alltesty = load_usps("data/USPS_test.txt")


def load_one_class(number):
    X_train, y_train = get_usps(number, alltrainx, alltrainy)
    X_test, y_test = get_usps(number, alltestx, alltesty)
    y_train = np.where(y_train == number, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == number, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


def load_two_classes(neg, pos):
    X_train, y_train = get_usps([neg, pos], alltrainx, alltrainy)
    X_test, y_test = get_usps([neg, pos], alltestx, alltesty)
    y_train = np.where(y_train == neg, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == neg, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


In [3]:
X_train, y_train, X_test, y_test = load_two_classes(6, 9)


In [4]:
alltrainx.shape

(7291, 256)

In [5]:
import numpy as np
X_train[:,:,np.newaxis].shape

(1308, 256, 1)

In [6]:
conv = Conv1D(3, 1, 1)

In [7]:
conv.forward(X_train[:,:,np.newaxis]).shape

(1308, 254, 1)

## Vérif Conv

In [8]:
import torch
import torch.nn as nn


In [9]:
convtest = nn.Conv1d(3, 1, 3, stride=1, bias=False)
# chan_in, chan_out, k_size

In [10]:
input = torch.randn(20, 3, 50) # N, C_IN, L_IN
convtest(input).shape

torch.Size([20, 1, 48])

In [11]:
input.numpy().shape

(20, 3, 50)

In [12]:
np.transpose(convtest.weight.data.numpy()).shape

(3, 3, 1)

In [13]:
conv = Conv1D(3, 3, 1, stride=1)
conv._chan_in

3

In [14]:
t = conv.forward(input.numpy().reshape((20, 50, 3)))

In [15]:
t.shape

(20, 48, 1)

## Vérif MaxPool

In [16]:
maxpool = MaxPool1D(2, 2)
t = maxpool.forward(input.numpy().reshape((20, 50, 3)))
t

array([[[1.00785136, 1.00785136, 1.00785136],
        [0.90141606, 0.90141606, 0.90141606],
        [0.91097325, 0.91097325, 0.91097325],
        ...,
        [1.44450426, 1.44450426, 1.44450426],
        [1.28324807, 1.28324807, 1.28324807],
        [1.05313575, 1.05313575, 1.05313575]],

       [[1.86798263, 1.86798263, 1.86798263],
        [2.99563146, 2.99563146, 2.99563146],
        [2.4353559 , 2.4353559 , 2.4353559 ],
        ...,
        [1.24657381, 1.24657381, 1.24657381],
        [1.14023805, 1.14023805, 1.14023805],
        [0.79479754, 0.79479754, 0.79479754]],

       [[1.50867891, 1.50867891, 1.50867891],
        [2.37552953, 2.37552953, 2.37552953],
        [0.21415713, 0.21415713, 0.21415713],
        ...,
        [1.37946522, 1.37946522, 1.37946522],
        [2.02472448, 2.02472448, 2.02472448],
        [0.78938854, 0.78938854, 0.78938854]],

       ...,

       [[1.66047263, 1.66047263, 1.66047263],
        [0.51386803, 0.51386803, 0.51386803],
        [1.21378493, 1

In [17]:
maxpooltorch = nn.MaxPool1d(2, 2)
maxpooltorch(input).shape

torch.Size([20, 3, 25])

In [18]:
AvgPool1D(2, 2).forward(input.numpy().reshape((20, 50, 3))).shape

(20, 25, 3)

In [23]:
t = Flatten()
t.forward(input.numpy().reshape((20, 50, 3)))

array([[ 0.86291444,  1.0078514 ,  0.67324406, ..., -0.2007521 ,
         0.82225597, -0.9596776 ],
       [ 1.0161036 ,  0.73569316,  0.96725166, ..., -0.27027583,
        -2.8135114 , -0.9865924 ],
       [ 1.5086789 ,  0.75132716, -0.38362893, ..., -0.85508835,
         0.48959982, -0.12163543],
       ...,
       [-1.3150572 ,  0.31165445, -0.6139964 , ...,  0.45809793,
        -0.5672247 ,  0.6660352 ],
       [ 0.36484647, -0.2129462 ,  0.27619967, ..., -0.42395306,
         1.5194488 ,  0.6684072 ],
       [-1.8968289 ,  1.5200917 ,  0.42094302, ..., -0.54694444,
        -0.30943432, -0.16444738]], dtype=float32)

In [25]:
t.backward(input.numpy().reshape((20, 50, 3))).shape

(20, 50, 3)